# Invoke-SqlNotebook
Your first option for automating SQL Notebooks is the Invoke-SqlNotebook cmdlet.

In [2]:
Import-Module SqlServer;

The `Invoke-SqlNotebook` is fairly straightforward to use.  There are three things you want to tell it:

+ Which SQL Notebook file you want to use
+ Which SQL Server database you want to connect to
+ What you want to name of your results-Notbook to be (If you don't specify a name, it will echo the Notebook's name, and add `-Results` to the end)

In [ ]:
Invoke-SqlNotebook

Download the Best Practices Check Notebook from the Tiger Team's GitHub repo.

<img src="https://raw.githubusercontent.com/SQLvariant/sqlworkshops/Notebooks-demo-ThrowAway/graphics/Tiger%20Logo%201.png" width="10.65%">

In [4]:
irm https://raw.githubusercontent.com/microsoft/tigertoolbox/master/BPCheck/BPCheck.ipynb -OutFile "$home\Downloads\BPCheck.ipynb"

                                                                                                    
 Web request status                                                                                 
    Number of bytes processed: 0                                                                    
                                                                                                    

# I heard you like to run Notebooks

## So I created a PowerShell Notebook to execute a T-SQL Notebook, using the new Invoke-SqlNotebook PowerShell cmdlet

This is a simple demonstration of how your can use a PowerShell Notebook to execute a T-SQL Notebook across all your Registered SQL Servers, using the new Invoke-SqlNotebook cmdlet. The results of the execution against each SQL Server Instance will be saved locally to it's own Output-Notebook, with a unique name.

In [3]:
dir 'SQLSERVER:\SQLRegistration\Database Engine Server Group'

    Directory: 


    Microsoft.SqlServer.Management.PSProvider\SqlServer::SQLSERVER:\SQLRegistration\Database 


    Engine Server Group


Mode Name


---- ----


d    Dockers


-    localhost\SQL2016


-    localhost\SQL2016Happy


-    localhost\SQL2017


-    localhost\SQL2019


In [15]:
dir 'SQLSERVER:\SQLRegistration\Database Engine Server Group' |
WHERE { $_.Mode -ne 'd'} |
foreach {
    Get-SqlInstance -ServerInstance $_.Name
        }

Instance Name                   Version    ProductLevel UpdateLevel  HostPlatform HostDistribution


-------------                   -------    ------------ -----------  ------------ ----------------


localhost\SQL2016               13.0.1742  RTM          n/a          Windows      n/a


localhost\SQL2016Happy          13.0.5622  SP2          CU11         Windows      n/a


localhost\SQL2017               14.0.3192  RTM          CU15         Windows      Windows 10 Pro


localhost\SQL2019               15.0.2070  RTM          n/a          Windows      Windows 10 Pro


Build the loop one step deeper, just using the `Get-SqlDatabase` as a placeholder to make sure things work before we switch over to using the `Invoke-SqlNotebook` cmdlet.

In [ ]:
dir 'SQLSERVER:\SQLRegistration\Database Engine Server Group' |
WHERE { $_.Mode -ne 'd'} |            
foreach {
    Get-SqlInstance -ServerInstance $_.Name |
    foreach {
            Get-SqlDatabase -ServerInstance $_.Name
            }
        }

OK, now that we've verified we can talk to the instances, we're finally ready to run this loop using a Notebook & record the results.

In [14]:
dir 'SQLSERVER:\SQLRegistration\Database Engine Server Group' |
WHERE { $_.Mode -ne 'd'} |
foreach {
    Get-SqlInstance -ServerInstance $_.Name |
    foreach {
            Invoke-SqlNotebook -ServerInstance $_.Name -Database master -InputFile "$home\Downloads\BPCheck.ipynb" `
            -OutputFile "c:\temp\BPChecks\BPCheck_output_$($_.NetName)_$($_.InstanceName)_$(Get-Date -Format yyyyMMdd).ipynb"
            }
        }

    Directory: C:\temp\BPChecks


Mode                 LastWriteTime         Length Name


----                 -------------         ------ ----


-a---           9/16/2020 10:56 AM         526714 BPCheck_output_SURFACE215SQL_SQL2016_20200916.ipy


                                                  nb


...

                                                  6.ipynb


-a---           9/16/2020 10:56 AM         655852 BPCheck_output_SURFACE215SQL_SQL2017_20200916.ipy


                                                  nb


-a---           9/16/2020 10:56 AM         510847 BPCheck_output_SURFACE215SQL_SQL2019_20200916.ipy


                                                  nb


## Same progression as before, but with Docker & Credentials

In [8]:
# Use this one for Docker containers.
$SQLDockCred = Get-Credential sa

dir 'SQLSERVER:\SQLRegistration\Database Engine Server Group\Dockers' |
WHERE { $_.Mode -ne 'd'} |
foreach {
    Get-SqlInstance -ServerInstance $_.Name -Credential $SQLDockCred 
        }



Name                 Status           Size     Space  Recovery Compat. Owner                     Collation             
                                            Available  Model     Level                                                 
----                 ------           ---- ---------- -------- ------- -----                     ---------             
AdventureWorks2016   Normal      209.63 MB    1.13 MB Simple       130 sa                        SQL_Latin1_General_CP1
                                                                                                 _CI_AS                
AdventureWorksDW2012 Normal      167.00 MB   32.47 MB Simple       110 sa                        SQL_Latin1_General_CP1
                                                                                                 _CI_AS                
AdventureWorksDW2014 Normal      188.00 MB   78.10 MB Simple       120 sa                        SQL_Latin1_General_CP1
                                       

In [7]:
$SQLDockCred = Get-Credential sa

dir 'SQLSERVER:\SQLRegistration\Database Engine Server Group\Dockers' |
WHERE { $_.Mode -ne 'd'} |
foreach {
    Get-SqlInstance -ServerInstance $_.Name -Credential $SQLDockCred |
    foreach {
            Get-SqlDatabase -ServerInstance $_.Name -Credential $SQLDockCred
            }
        }


& { Import-Module SqlServer;
>> 
>> $SQLDockCred = Get-Credential sa
>> 
>> dir 'SQLSERVER:\SQLRegistration\Database Engine Server Group' |
>> WHERE { $_.Mode -ne 'd'} |            
>> foreach {
>>     Get-SqlInstance -ServerInstance $_.Name -Credential $SQLDockCred |
>>     foreach {
>>             Get-SqlDatabase -ServerInstance $_.Name -Credential $SQLDockCred
>>             }
>>         }
>>  }
>> 

Name                 Status           Size     Space  Recovery Compat. Owner                     Collation             
                                            Available  Model     Level                                                 
----                 ------           ---- ---------- -------- ------- -----                     ---------             
AdventureWorks2016   Normal      209.63 MB    1.13 MB Simple       130 sa                        SQL_Latin1_General_CP1
                                                                                                 _CI_AS          

In [4]:
Import-Module SqlServer;

$SQLDockCred = Get-Credential sa

dir 'SQLSERVER:\SQLRegistration\Database Engine Server Group\Dockers' |
WHERE { $_.Mode -ne 'd'} |
foreach {
    Get-SqlInstance -ServerInstance $_.Name -Credential $SQLDockCred |
    foreach {
            Invoke-SqlNotebook -ServerInstance $_.Name -Database master -Credential $SQLDockCred -InputFile "$home\Downloads\BPCheck.ipynb" `
            -OutputFile "c:\temp\BPChecks\BPCheck_output_$($_.NetName)_$($_.InstanceName)_$(Get-Date -Format yyyyMMdd).ipynb"
            }
        }